In [1]:
# auto-reload modules
%load_ext autoreload
%autoreload 2

In [43]:
from Value import *

In [47]:
a = Value(8)
b = Value(9)
c = a + b

c.backward()
print(c.grad, b.grad, a.grad)

1 1 1


In [49]:
d = a * b
d.backward()
print(d.grad, b.grad, a.grad)

1 8 9
